# 1- Driving_data.py

In [4]:
import cv2
import random
import scipy
# from skimage.transform import resize

xs = []
ys = []

#points to the end of the last batch
train_batch_pointer = 0
val_batch_pointer = 0

#read data.txt
with open("../data.txt") as f:
    for line in f: 
        xs.append("../data/" + line.split()[0])
        #the paper by Nvidia uses the inverse of the turning radius,
        #but steering wheel angle is proportional to the inverse of turning radius
        #so the steering wheel angle in radians is used as the output
        ys.append(float(line.split(",")[0].split()[1]) * scipy.pi / 180)

#get number of images
num_images = len(xs)


train_xs = xs[:int(len(xs) * 0.8)]
train_ys = ys[:int(len(xs) * 0.8)]

val_xs = xs[-int(len(xs) * 0.2):]
val_ys = ys[-int(len(xs) * 0.2):]

num_train_images = len(train_xs)
num_val_images = len(val_xs)

def LoadTrainBatch(batch_size):
    global train_batch_pointer
    x_out = []
    y_out = []
    for i in range(0, batch_size):
        x_out.append(cv2.resize(cv2.imread(train_xs[(train_batch_pointer + i) % num_train_images])[-150:], (66,200)) / 255.0)
        y_out.append([train_ys[(train_batch_pointer + i) % num_train_images]])
    train_batch_pointer += batch_size
    return x_out, y_out

def LoadValBatch(batch_size):
    global val_batch_pointer
    x_out = []
    y_out = []
    for i in range(0, batch_size):
        x_out.append(cv2.resize(cv2.imread(val_xs[(val_batch_pointer + i) % num_val_images])[-150:], [66, 200]) / 255.0)
        y_out.append([val_ys[(val_batch_pointer + i) % num_val_images]])
    val_batch_pointer += batch_size
    return x_out, y_out


# 2-model.py

In [2]:
import tensorflow as tf
import scipy

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W, stride):
    return tf.nn.conv2d(x, W, strides=[1, stride, stride, 1], padding='VALID')

x = tf.placeholder(tf.float32, shape=[None, 66, 200, 3])
y_ = tf.placeholder(tf.float32, shape=[None, 1])

x_image = x

#first convolutional layer
W_conv1 = weight_variable([5, 5, 3, 24])
b_conv1 = bias_variable([24])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1, 2) + b_conv1)

#second convolutional layer
W_conv2 = weight_variable([5, 5, 24, 36])
b_conv2 = bias_variable([36])

h_conv2 = tf.nn.relu(conv2d(h_conv1, W_conv2, 2) + b_conv2)

#third convolutional layer
W_conv3 = weight_variable([5, 5, 36, 48])
b_conv3 = bias_variable([48])

h_conv3 = tf.nn.relu(conv2d(h_conv2, W_conv3, 2) + b_conv3)

#fourth convolutional layer
W_conv4 = weight_variable([3, 3, 48, 64])
b_conv4 = bias_variable([64])

h_conv4 = tf.nn.relu(conv2d(h_conv3, W_conv4, 1) + b_conv4)

#fifth convolutional layer
W_conv5 = weight_variable([3, 3, 64, 64])
b_conv5 = bias_variable([64])

h_conv5 = tf.nn.relu(conv2d(h_conv4, W_conv5, 1) + b_conv5)

#FCL 1
W_fc1 = weight_variable([1152, 1164])
b_fc1 = bias_variable([1164])

h_conv5_flat = tf.reshape(h_conv5, [-1, 1152])
h_fc1 = tf.nn.relu(tf.matmul(h_conv5_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

#FCL 2
W_fc2 = weight_variable([1164, 100])
b_fc2 = bias_variable([100])

h_fc2 = tf.nn.relu(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

h_fc2_drop = tf.nn.dropout(h_fc2, keep_prob)

#FCL 3
W_fc3 = weight_variable([100, 50])
b_fc3 = bias_variable([50])

h_fc3 = tf.nn.relu(tf.matmul(h_fc2_drop, W_fc3) + b_fc3)

h_fc3_drop = tf.nn.dropout(h_fc3, keep_prob)

#FCL 3
W_fc4 = weight_variable([50, 10])
b_fc4 = bias_variable([10])

h_fc4 = tf.nn.relu(tf.matmul(h_fc3_drop, W_fc4) + b_fc4)

h_fc4_drop = tf.nn.dropout(h_fc4, keep_prob)

#Output
W_fc5 = weight_variable([10, 1])
b_fc5 = bias_variable([1])

y = tf.matmul(h_fc4_drop, W_fc5) + b_fc5  #scale the atan output


C:\Users\hp\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# 3- Train.py

In [3]:
import os
import tensorflow as tf
from tensorflow.core.protobuf import saver_pb2
import driving_data
import model

LOGDIR = './save'

sess = tf.InteractiveSession()

L2NormConst = 0.001

train_vars = tf.trainable_variables()

loss = tf.reduce_mean(tf.square(tf.subtract(y_, y))) + tf.add_n([tf.nn.l2_loss(v) for v in train_vars]) * L2NormConst
train_step = tf.train.AdamOptimizer(1e-3).minimize(loss)
sess.run(tf.initialize_all_variables())

# create a summary to monitor cost tensor
tf.summary.scalar("loss", loss)
# merge all summaries into a single op
merged_summary_op =  tf.summary.merge_all()

saver = tf.train.Saver(write_version = saver_pb2.SaverDef.V1)

# op to write logs to Tensorboard
logs_path = './logs'
summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

epochs = 30
batch_size = 100

# train over the dataset about 30 times
for epoch in range(epochs):
    for i in range(int(driving_data.num_images/batch_size)):
        xs, ys = driving_data.LoadTrainBatch(batch_size)
        train_step.run(feed_dict={x: xs,y_: ys, keep_prob: 0.5})
        if i % 10 == 0:
            xs, ys = driving_data.LoadValBatch(batch_size)
            loss_value = loss.eval(feed_dict={x:xs, y_: ys, keep_prob: 1.0})
            print("Epoch: %d, Step: %d, Loss: %g" % (epoch, epoch * batch_size + i, loss_value))

    # write logs at every iteration
        summary = merged_summary_op.eval(feed_dict={x:xs, y_: ys, keep_prob: 1.0})
        summary_writer.add_summary(summary, epoch * driving_data.num_images/batch_size + i)
  
        if i % batch_size == 0:
            if not os.path.exists(LOGDIR):
                os.makedirs(LOGDIR)
            checkpoint_path = os.path.join(LOGDIR, "model.ckpt")
            filename = saver.save(sess, checkpoint_path)
    print("Model saved in file: %s" % filename)

print("Run the command line:\n" \
          "--> tensorboard --logdir=./logs " \
          "\nThen open http://0.0.0.0:6006/ into your web browser")


Instructions for updating:
Use `tf.global_variables_initializer` instead.
Epoch: 0, Step: 0, Loss: 12.5028
Epoch: 0, Step: 10, Loss: 11.1882
Epoch: 0, Step: 20, Loss: 9.55802
Epoch: 0, Step: 30, Loss: 8.30127
Epoch: 0, Step: 40, Loss: 7.31824
Epoch: 0, Step: 50, Loss: 6.52161
Epoch: 0, Step: 60, Loss: 5.97318
Epoch: 0, Step: 70, Loss: 6.19692
Epoch: 0, Step: 80, Loss: 6.33867
Epoch: 0, Step: 90, Loss: 4.5621
Epoch: 0, Step: 100, Loss: 4.06487
Epoch: 0, Step: 110, Loss: 3.667
Epoch: 0, Step: 120, Loss: 3.40036
Epoch: 0, Step: 130, Loss: 3.17038
Epoch: 0, Step: 140, Loss: 2.97752
Epoch: 0, Step: 150, Loss: 2.83743
Epoch: 0, Step: 160, Loss: 2.67296
Epoch: 0, Step: 170, Loss: 2.48287
Epoch: 0, Step: 180, Loss: 2.45352
Epoch: 0, Step: 190, Loss: 2.61562
Epoch: 0, Step: 200, Loss: 2.20557
Epoch: 0, Step: 210, Loss: 2.19193
Epoch: 0, Step: 220, Loss: 1.96089
Epoch: 0, Step: 230, Loss: 1.88555
Epoch: 0, Step: 240, Loss: 1.81542
Epoch: 0, Step: 250, Loss: 1.74994
Epoch: 0, Step: 260, Loss: 1.7

Epoch: 2, Step: 230, Loss: 0.356159
Epoch: 2, Step: 240, Loss: 0.327479
Epoch: 2, Step: 250, Loss: 0.434207
Epoch: 2, Step: 260, Loss: 1.35569
Epoch: 2, Step: 270, Loss: 1.61393
Epoch: 2, Step: 280, Loss: 0.357496
Epoch: 2, Step: 290, Loss: 0.404162
Epoch: 2, Step: 300, Loss: 0.278669
Epoch: 2, Step: 310, Loss: 0.276736
Epoch: 2, Step: 320, Loss: 0.275874
Epoch: 2, Step: 330, Loss: 0.276726
Epoch: 2, Step: 340, Loss: 0.298487
Epoch: 2, Step: 350, Loss: 0.278481
Epoch: 2, Step: 360, Loss: 0.252936
Epoch: 2, Step: 370, Loss: 0.371919
Epoch: 2, Step: 380, Loss: 0.664482
Epoch: 2, Step: 390, Loss: 0.382598
Epoch: 2, Step: 400, Loss: 0.331024
Epoch: 2, Step: 410, Loss: 0.236494
Epoch: 2, Step: 420, Loss: 0.232342
Epoch: 2, Step: 430, Loss: 0.227408
Epoch: 2, Step: 440, Loss: 0.222315
Epoch: 2, Step: 450, Loss: 0.279167
Epoch: 2, Step: 460, Loss: 5.51246
Epoch: 2, Step: 470, Loss: 0.388251
Epoch: 2, Step: 480, Loss: 0.209396
Epoch: 2, Step: 490, Loss: 0.199632
Epoch: 2, Step: 500, Loss: 0.20

Epoch: 4, Step: 520, Loss: 0.0945628
Epoch: 4, Step: 530, Loss: 0.132324
Epoch: 4, Step: 540, Loss: 0.073936
Epoch: 4, Step: 550, Loss: 0.0853868
Epoch: 4, Step: 560, Loss: 0.190549
Epoch: 4, Step: 570, Loss: 0.464456
Epoch: 4, Step: 580, Loss: 0.290823
Epoch: 4, Step: 590, Loss: 0.0786301
Epoch: 4, Step: 600, Loss: 0.0730066
Epoch: 4, Step: 610, Loss: 0.0724672
Epoch: 4, Step: 620, Loss: 0.0717979
Epoch: 4, Step: 630, Loss: 0.0708623
Epoch: 4, Step: 640, Loss: 0.540946
Epoch: 4, Step: 650, Loss: 4.98272
Epoch: 4, Step: 660, Loss: 0.101716
Epoch: 4, Step: 670, Loss: 0.0674729
Epoch: 4, Step: 680, Loss: 0.0631511
Epoch: 4, Step: 690, Loss: 0.0686472
Epoch: 4, Step: 700, Loss: 0.0636819
Epoch: 4, Step: 710, Loss: 0.0610029
Epoch: 4, Step: 720, Loss: 0.0550996
Epoch: 4, Step: 730, Loss: 0.0545147
Epoch: 4, Step: 740, Loss: 0.0537855
Epoch: 4, Step: 750, Loss: 0.0525925
Epoch: 4, Step: 760, Loss: 0.0545566
Epoch: 4, Step: 770, Loss: 0.0861623
Epoch: 4, Step: 780, Loss: 0.0900766
Epoch: 4, 

Epoch: 6, Step: 810, Loss: 0.0334161
Epoch: 6, Step: 820, Loss: 0.0363042
Epoch: 6, Step: 830, Loss: 1.54568
Epoch: 6, Step: 840, Loss: 3.99648
Epoch: 6, Step: 850, Loss: 0.0360902
Epoch: 6, Step: 860, Loss: 0.0253262
Epoch: 6, Step: 870, Loss: 0.0252619
Epoch: 6, Step: 880, Loss: 0.0299467
Epoch: 6, Step: 890, Loss: 0.0254144
Epoch: 6, Step: 900, Loss: 0.0258421
Epoch: 6, Step: 910, Loss: 0.0232962
Epoch: 6, Step: 920, Loss: 0.022989
Epoch: 6, Step: 930, Loss: 0.0234607
Epoch: 6, Step: 940, Loss: 0.0203954
Epoch: 6, Step: 950, Loss: 0.0253222
Epoch: 6, Step: 960, Loss: 0.0584225
Epoch: 6, Step: 970, Loss: 0.0575175
Epoch: 6, Step: 980, Loss: 0.0273087
Epoch: 6, Step: 990, Loss: 0.0190069
Epoch: 6, Step: 1000, Loss: 0.0193447
Epoch: 6, Step: 1010, Loss: 0.0194111
Epoch: 6, Step: 1020, Loss: 0.0185048
Epoch: 6, Step: 1030, Loss: 0.0191244
Epoch: 6, Step: 1040, Loss: 0.0182741
Epoch: 6, Step: 1050, Loss: 0.0174577
Model saved in file: ./save\model.ckpt
Epoch: 7, Step: 700, Loss: 0.018669

Epoch: 8, Step: 1110, Loss: 0.00929671
Epoch: 8, Step: 1120, Loss: 0.0099365
Epoch: 8, Step: 1130, Loss: 0.00787809
Epoch: 8, Step: 1140, Loss: 0.0182564
Epoch: 8, Step: 1150, Loss: 0.0550452
Epoch: 8, Step: 1160, Loss: 0.0482509
Epoch: 8, Step: 1170, Loss: 0.00810573
Epoch: 8, Step: 1180, Loss: 0.00769767
Epoch: 8, Step: 1190, Loss: 0.00805867
Epoch: 8, Step: 1200, Loss: 0.00818513
Epoch: 8, Step: 1210, Loss: 0.00752289
Epoch: 8, Step: 1220, Loss: 0.00965413
Epoch: 8, Step: 1230, Loss: 0.00675299
Epoch: 8, Step: 1240, Loss: 0.00739919
Epoch: 8, Step: 1250, Loss: 0.00973865
Model saved in file: ./save\model.ckpt
Epoch: 9, Step: 900, Loss: 0.00795179
Epoch: 9, Step: 910, Loss: 0.00888922
Epoch: 9, Step: 920, Loss: 0.00755001
Epoch: 9, Step: 930, Loss: 0.0074207
Epoch: 9, Step: 940, Loss: 0.00766839
Epoch: 9, Step: 950, Loss: 0.00824656
Epoch: 9, Step: 960, Loss: 0.0420644
Epoch: 9, Step: 970, Loss: 0.0388428
Epoch: 9, Step: 980, Loss: 5.01068
Epoch: 9, Step: 990, Loss: 0.464467
Epoch: 9

...

Epoch: 21, Step: 2310, Loss: 0.0193927
Epoch: 21, Step: 2320, Loss: 0.0203632
Epoch: 21, Step: 2330, Loss: 0.0208274
Epoch: 21, Step: 2340, Loss: 0.0211852
Epoch: 21, Step: 2350, Loss: 0.0206158
Epoch: 21, Step: 2360, Loss: 0.0249357
Epoch: 21, Step: 2370, Loss: 0.0310477
Epoch: 21, Step: 2380, Loss: 0.0385458
Epoch: 21, Step: 2390, Loss: 0.00874352
Epoch: 21, Step: 2400, Loss: 0.0203525
Epoch: 21, Step: 2410, Loss: 0.084879
Epoch: 21, Step: 2420, Loss: 0.0248376
Epoch: 21, Step: 2430, Loss: 0.33172
Epoch: 21, Step: 2440, Loss: 0.618903
Epoch: 21, Step: 2450, Loss: 0.240205
Epoch: 21, Step: 2460, Loss: 0.0727394
Epoch: 21, Step: 2470, Loss: 0.0408338
Epoch: 21, Step: 2480, Loss: 0.0145943
Epoch: 21, Step: 2490, Loss: 0.120888
Epoch: 21, Step: 2500, Loss: 0.938268
Epoch: 21, Step: 2510, Loss: 1.42445
Epoch: 21, Step: 2520, Loss: 0.117922
Epoch: 21, Step: 2530, Loss: 0.120697
Epoch: 21, Step: 2540, Loss: 0.00131792
Epoch: 21, Step: 2550, Loss: 0.00585507
Model saved in file: ./save\model

Epoch: 23, Step: 2540, Loss: 0.0166806
Epoch: 23, Step: 2550, Loss: 0.0284338
Epoch: 23, Step: 2560, Loss: 0.0238155
Epoch: 23, Step: 2570, Loss: 0.0331834
Epoch: 23, Step: 2580, Loss: 0.0048106
Epoch: 23, Step: 2590, Loss: 0.0285908
Epoch: 23, Step: 2600, Loss: 0.0930695
Epoch: 23, Step: 2610, Loss: 0.0143288
Epoch: 23, Step: 2620, Loss: 0.449415
Epoch: 23, Step: 2630, Loss: 0.611322
Epoch: 23, Step: 2640, Loss: 0.205935
Epoch: 23, Step: 2650, Loss: 0.04693
Epoch: 23, Step: 2660, Loss: 0.0376843
Epoch: 23, Step: 2670, Loss: 0.0165816
Epoch: 23, Step: 2680, Loss: 0.146529
Epoch: 23, Step: 2690, Loss: 1.16193
Epoch: 23, Step: 2700, Loss: 1.18265
Epoch: 23, Step: 2710, Loss: 0.0531732
Epoch: 23, Step: 2720, Loss: 0.107128
Epoch: 23, Step: 2730, Loss: 0.00216327
Epoch: 23, Step: 2740, Loss: 0.00434543
Epoch: 23, Step: 2750, Loss: 0.00941206
Model saved in file: ./save\model.ckpt
Epoch: 24, Step: 2400, Loss: 0.0182498
Epoch: 24, Step: 2410, Loss: 0.0493964
Epoch: 24, Step: 2420, Loss: 0.01

Epoch: 25, Step: 2810, Loss: 0.544828
Epoch: 25, Step: 2820, Loss: 0.547135
Epoch: 25, Step: 2830, Loss: 0.156539
Epoch: 25, Step: 2840, Loss: 0.0370333
Epoch: 25, Step: 2850, Loss: 0.023
Epoch: 25, Step: 2860, Loss: 0.0371639
Epoch: 25, Step: 2870, Loss: 0.281221
Epoch: 25, Step: 2880, Loss: 1.42065
Epoch: 25, Step: 2890, Loss: 0.885916
Epoch: 25, Step: 2900, Loss: 0.0676058
Epoch: 25, Step: 2910, Loss: 0.0561349
Epoch: 25, Step: 2920, Loss: 0.00406897
Epoch: 25, Step: 2930, Loss: 0.00367001
Epoch: 25, Step: 2940, Loss: 0.00945858
Epoch: 25, Step: 2950, Loss: 0.0265574
Model saved in file: ./save\model.ckpt
Epoch: 26, Step: 2600, Loss: 0.0617257
Epoch: 26, Step: 2610, Loss: 0.00219065
Epoch: 26, Step: 2620, Loss: 0.0363583
Epoch: 26, Step: 2630, Loss: 0.138225
Epoch: 26, Step: 2640, Loss: 0.381552
Epoch: 26, Step: 2650, Loss: 0.226084
Epoch: 26, Step: 2660, Loss: 0.0178889
Epoch: 26, Step: 2670, Loss: 0.0125151
Epoch: 26, Step: 2680, Loss: 0.0131172
Epoch: 26, Step: 2690, Loss: 0.0136

Epoch: 27, Step: 3080, Loss: 0.604678
Epoch: 27, Step: 3090, Loss: 0.108695
Epoch: 27, Step: 3100, Loss: 0.0226487
Epoch: 27, Step: 3110, Loss: 0.00592061
Epoch: 27, Step: 3120, Loss: 0.00100612
Epoch: 27, Step: 3130, Loss: 0.00960139
Epoch: 27, Step: 3140, Loss: 0.0297582
Epoch: 27, Step: 3150, Loss: 0.0528889
Model saved in file: ./save\model.ckpt
Epoch: 28, Step: 2800, Loss: 0.00220907
Epoch: 28, Step: 2810, Loss: 0.0738561
Epoch: 28, Step: 2820, Loss: 0.154284
Epoch: 28, Step: 2830, Loss: 0.343746
Epoch: 28, Step: 2840, Loss: 0.211544
Epoch: 28, Step: 2850, Loss: 0.0132407
Epoch: 28, Step: 2860, Loss: 0.0103053
Epoch: 28, Step: 2870, Loss: 0.0102835
Epoch: 28, Step: 2880, Loss: 0.00977382
Epoch: 28, Step: 2890, Loss: 0.0161215
Epoch: 28, Step: 2900, Loss: 2.14197
Epoch: 28, Step: 2910, Loss: 3.35507
Epoch: 28, Step: 2920, Loss: 0.0140848
Epoch: 28, Step: 2930, Loss: 0.00470452
Epoch: 28, Step: 2940, Loss: 0.004887
Epoch: 28, Step: 2950, Loss: 0.0120926
Epoch: 28, Step: 2960, Loss: 

Epoch: 29, Step: 3310, Loss: 0.00282961
Epoch: 29, Step: 3320, Loss: 0.012246
Epoch: 29, Step: 3330, Loss: 0.0390101
Epoch: 29, Step: 3340, Loss: 0.048629
Epoch: 29, Step: 3350, Loss: 0.00562949
Model saved in file: ./save\model.ckpt
Run the command line:
--> tensorboard --logdir=./logs 
Then open http://0.0.0.0:6006/ into your web browser


# Torch

In [71]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils import data
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

class NetworkDense(nn.Module):

    def __init__(self):
        super(NetworkDense, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 24, 5, stride=2),
            nn.ELU(),
            nn.Conv2d(24, 36, 5, stride=2),
            nn.ELU(),
            nn.Conv2d(36, 48, 5, stride=2),
            nn.ELU(),
            nn.Conv2d(48, 64, 3),
            nn.ELU(),
            nn.Conv2d(64, 64, 3),
            nn.Dropout(0.25)
        )
        self.linear_layers = nn.Sequential(
            nn.Linear(in_features=64 * 2 * 33, out_features=100),
            nn.ELU(),
            nn.Linear(in_features=100, out_features=50),
            nn.ELU(),
            nn.Linear(in_features=50, out_features=10),
            nn.Linear(in_features=10, out_features=1)
        )
        
    def forward(self, input):
        input = input.view(100, 66, 200, 3)
        output = self.conv_layers(input)
        print(output.shape)
        output = output.view(output.size(0), -1)
        output = self.linear_layers(output)
        return output

In [72]:
model = NetworkDense()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

criterion = nn.MSELoss()

In [73]:
# Step6: Check the device and define function to move tensors to that device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 
print('device is: ', device)

def toDevice(imgs, angles, device):
    return imgs.float().to(device), angles.float().to(device)

device is:  cuda


In [80]:
import driving_data
max_epochs = 22
batch_size = 100

for epoch in range(max_epochs):
#     model.to(device)
    
    # Training
    train_loss = 0
    model.train()
#     for local_batch, (centers, lefts, rights) in enumerate(training_generator):
    for i in range(int(driving_data.num_images/batch_size)):
        # Transfer to GPU
        xs, ys = driving_data.LoadTrainBatch(batch_size)
        xs = np.array(xs)
        ys = np.array(ys)
        
#         xs, ys = toDevice(xs,ys, device)
        
        # Model computations
        optimizer.zero_grad()
#         datas = [xs,ys]        
#         for data in datas:
        imgs, angles = xs, ys
#             print("training image: ", imgs.shape)
        outputs = model(imgs)
        loss = criterion(outputs, angles.unsqueeze(1))
        loss.backward()
        optimizer.step()

        train_loss += loss.data[0].item()

    if local_batch % 100 == 0:
        print('Loss: %.3f '
            % (train_loss/(local_batch+1)))

TypeError: view() takes at most 2 arguments (4 given)

In [85]:
imgs.size

3960000

In [77]:
imgs.view(100, 66, 200, 3)

TypeError: Field elements must be 2- or 3-tuples, got '100'

# 4- run_dataset.py

In [1]:
#pip3 install opencv-python

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import scipy.misc
import model
import cv2
from subprocess import call
import math

# tf.disable_v2_behavior()
# sess = tf.InteractiveSession()
# saver = tf.train.Saver()

sess=tf.compat.v1.InteractiveSession()
saver = tf.compat.v1.train.Saver()
saver.restore(sess, "./model.ckpt")

img = cv2.imread('steering_wheel_image.jpg',0)
rows,cols = img.shape

smoothed_angle = 0


#read data.txt
xs = []
ys = []
with open("../data.txt") as f:
    for line in f:
        xs.append("../data/" + line.split()[0])
        #the paper by Nvidia uses the inverse of the turning radius,
        #but steering wheel angle is proportional to the inverse of turning radius
        #so the steering wheel angle in radians is used as the output
        ys.append(float(line.split(",")[0].split()[1]) * scipy.pi / 180)

#get number of images
num_images = len(xs)


i = math.ceil(num_images*0.7)
print("Starting frameofvideo:" +str(i))

while(cv2.waitKey(10) != ord('q')):
    full_image = cv2.imread("../data/" + str(i) + ".jpg")
#     image = scipy.misc.imresize(full_image[-150:], [66, 200]) / 255.0
    image = cv2.resize(full_image[-150:], (200, 66)) / 255.0
    degrees = model.y.eval(feed_dict={model.x: [image], model.keep_prob: 1.0})[0][0] * 180.0 / scipy.pi
    #call("clear")
    #print("Predicted Steering angle: " + str(degrees))
    print("Steering angle: " + str(degrees) + " (pred)\t" + str(ys[i]*180/scipy.pi) + " (actual)")
    cv2.imshow("frame", cv2.cvtColor(full_image, cv2.COLOR_RGB2BGR))
    #make smooth angle transitions by turning the steering wheel based on the difference of the current angle
    #and the predicted angle
    smoothed_angle += 0.2 * pow(abs((degrees - smoothed_angle)), 2.0 / 3.0) * (degrees - smoothed_angle) / abs(degrees - smoothed_angle)
    M = cv2.getRotationMatrix2D((cols/2,rows/2),-smoothed_angle,1)
    dst = cv2.warpAffine(img,M,(cols,rows))
    cv2.imshow("steering wheel", dst)
    i += 1

cv2.destroyAllWindows()


Instructions for updating:
non-resource variables are not supported in the long term
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
INFO:tensorflow:Restoring parameters from ./model.ckpt
Starting frameofvideo:44678
Steering angle: -2.4208038838469323 (pred)	0.1 (actual)
Steering angle: 7.494478854536204 (pred)	0.0 (actual)
Steering angle: 4.892276245391135 (pred)	0.1 (actual)
Steering angle: 1.7743230638106406 (pred)	0.1 (actual)
Steering angle: -0.08625065500335577 (pred)	0.1 (actual)
Steering angle: -1.0884250086307934 (pred)	0.1 (actual)
Steering angle: -1.6824913838632307 (pred)	0.1 (actual)
Steering angle: -2.380463292504956 (pred)	0.1 (actual)
Steering angle: 3.319206626465624 (pred)	0.1 (actual)
Steering angle: -0.295355388999202 (pred)	0.1 (actual)
Steering angle: -3.0147302402013842 (pred)	0.1 (actual)
Steering angle: -4.97117005294666 (pred)	0.1 (actual)
Steering angle: -3.3241023779966423 (pred)	0.1 (actual)

Steering angle: -1.5964552261482823 (pred)	0.2 (actual)
Steering angle: 0.923083631184533 (pred)	0.2 (actual)
Steering angle: -1.977694080781886 (pred)	0.2 (actual)
Steering angle: 0.48679990850382654 (pred)	0.2 (actual)
Steering angle: -2.969111406734869 (pred)	0.2 (actual)
Steering angle: -4.990510587355197 (pred)	0.2 (actual)
Steering angle: -6.98992552347137 (pred)	0.2 (actual)
Steering angle: -6.434853551984181 (pred)	0.2 (actual)
Steering angle: -9.84205329208367 (pred)	0.2 (actual)
Steering angle: -9.518093150881732 (pred)	0.2 (actual)
Steering angle: -9.129755915431488 (pred)	0.2 (actual)
Steering angle: -8.641426418079549 (pred)	0.2 (actual)
Steering angle: -7.9280507253121515 (pred)	0.2 (actual)
Steering angle: -6.8361339959116005 (pred)	0.2 (actual)
Steering angle: 3.901956018573585 (pred)	0.2 (actual)
Steering angle: 4.713334253760086 (pred)	0.2 (actual)
Steering angle: 0.6336512168149221 (pred)	0.2 (actual)
Steering angle: 0.5196878029659668 (pred)	0.2 (actual)
Steering an

Steering angle: 35.41592079001221 (pred)	0.0 (actual)
Steering angle: -9.595363080962088 (pred)	0.2 (actual)
Steering angle: -19.14924813064198 (pred)	0.2 (actual)
Steering angle: -15.605032234471098 (pred)	0.2 (actual)
Steering angle: -13.588091032944671 (pred)	0.2 (actual)
Steering angle: -15.62759917948882 (pred)	0.2 (actual)
Steering angle: -16.677998604019447 (pred)	0.2 (actual)
Steering angle: -12.225011524626384 (pred)	0.2 (actual)
Steering angle: -11.248299595956437 (pred)	0.2 (actual)
Steering angle: -8.889227388714904 (pred)	0.2 (actual)
Steering angle: -3.5653766759932273 (pred)	0.2 (actual)
Steering angle: -6.848780944933526 (pred)	0.2 (actual)
Steering angle: -8.100271383913084 (pred)	0.2 (actual)
Steering angle: -3.5761062628490827 (pred)	0.2 (actual)
Steering angle: -6.031600890933921 (pred)	0.2 (actual)
Steering angle: -5.156857710156742 (pred)	0.2 (actual)
Steering angle: -5.641054516174 (pred)	0.2 (actual)
Steering angle: -9.196278542852827 (pred)	0.2 (actual)
Steerin

Steering angle: -1.5393040451550237 (pred)	0.91 (actual)
Steering angle: -0.13339171352028462 (pred)	0.5 (actual)
Steering angle: 1.4421425764991826 (pred)	0.3 (actual)
Steering angle: 2.6871197897748997 (pred)	0.2 (actual)
Steering angle: 8.442712309461783 (pred)	0.2 (actual)
Steering angle: 12.032244803228407 (pred)	0.2 (actual)
Steering angle: 11.183875544157585 (pred)	0.0 (actual)
Steering angle: 23.608694007668937 (pred)	-1.6100000000000003 (actual)
Steering angle: 27.326234491765355 (pred)	-3.3299999999999996 (actual)
Steering angle: 23.137281181343827 (pred)	-3.7300000000000004 (actual)
Steering angle: 18.36170170613012 (pred)	-3.7300000000000004 (actual)
Steering angle: 8.72456177310965 (pred)	-3.03 (actual)
Steering angle: 5.522904623250651 (pred)	-2.7200000000000006 (actual)
Steering angle: 12.801722816064641 (pred)	-2.7200000000000006 (actual)
Steering angle: 25.75831163382286 (pred)	-2.7200000000000006 (actual)
Steering angle: 26.309362758323026 (pred)	-2.32 (actual)
Steeri

Steering angle: 15.476076555393965 (pred)	1.21 (actual)
Steering angle: 28.040764486502482 (pred)	1.21 (actual)
Steering angle: 27.80348371473624 (pred)	1.21 (actual)
Steering angle: 31.406114803783655 (pred)	1.21 (actual)
Steering angle: 32.64160723217975 (pred)	1.21 (actual)
Steering angle: 34.57654859762559 (pred)	1.21 (actual)
Steering angle: 35.89006569200533 (pred)	1.11 (actual)
Steering angle: 32.12506393091378 (pred)	1.11 (actual)
Steering angle: 7.796306621506361 (pred)	0.61 (actual)
Steering angle: -9.67477683666818 (pred)	0.5 (actual)
Steering angle: 17.011544061947852 (pred)	0.5 (actual)
Steering angle: 28.0639017523159 (pred)	0.0 (actual)
Steering angle: 35.43526388574169 (pred)	0.5 (actual)
Steering angle: 32.405088026505965 (pred)	0.5 (actual)
Steering angle: 33.60712618834734 (pred)	0.5 (actual)
Steering angle: 35.469466058010525 (pred)	0.5 (actual)
Steering angle: 31.917323727307846 (pred)	0.5 (actual)
Steering angle: 24.51508060109714 (pred)	0.5 (actual)
Steering angl

...

Steering angle: 77.91759593821155 (pred)	-14.22 (actual)
Steering angle: 82.64028925868183 (pred)	-14.62 (actual)
Steering angle: 70.65993937899354 (pred)	-16.639999000000003 (actual)
Steering angle: 53.56785201880561 (pred)	-18.66 (actual)
Steering angle: 37.19036978543567 (pred)	-20.370001 (actual)
Steering angle: 31.33760459131384 (pred)	-21.379999 (actual)
Steering angle: 12.34562754272693 (pred)	-22.790000999999997 (actual)
Steering angle: 0.19691255284268255 (pred)	-23.6 (actual)
Steering angle: 12.61458075544817 (pred)	-23.9 (actual)
Steering angle: 1.7463425535454487 (pred)	-23.9 (actual)
Steering angle: -1.590891610182484 (pred)	-23.9 (actual)
Steering angle: -13.08141308616114 (pred)	-23.9 (actual)
Steering angle: -12.374204200440618 (pred)	-23.9 (actual)
Steering angle: -15.312483279488404 (pred)	-23.290001 (actual)
Steering angle: -16.451712729269634 (pred)	-20.870001 (actual)
Steering angle: -15.027633655247607 (pred)	-19.870001 (actual)
Steering angle: -14.282392568681162

Steering angle: -21.543473613994276 (pred)	-4.64 (actual)
Steering angle: 86.63182448888061 (pred)	-4.64 (actual)
Steering angle: 122.38793309578907 (pred)	-4.64 (actual)
Steering angle: 127.35190967887922 (pred)	-4.64 (actual)
Steering angle: 114.57740687420439 (pred)	-4.64 (actual)
Steering angle: 106.96352862901354 (pred)	-4.64 (actual)
Steering angle: 112.77978987711623 (pred)	-4.54 (actual)
Steering angle: 115.37877930914364 (pred)	-4.34 (actual)
Steering angle: 120.7460239212097 (pred)	-4.13 (actual)
Steering angle: 121.47281069041242 (pred)	-3.9300000000000006 (actual)
Steering angle: 128.69777113407355 (pred)	0.0 (actual)
Steering angle: 129.75982456887553 (pred)	-3.829999999999999 (actual)
Steering angle: 127.26426469144963 (pred)	-3.829999999999999 (actual)
Steering angle: 101.14497926753889 (pred)	-3.829999999999999 (actual)
Steering angle: 76.02214331145213 (pred)	-3.829999999999999 (actual)
Steering angle: 56.50671849412201 (pred)	-3.829999999999999 (actual)
Steering angle

Steering angle: 32.421897122053366 (pred)	-7.36 (actual)
Steering angle: 3.371584572446102 (pred)	-7.36 (actual)
Steering angle: -21.93529595346852 (pred)	-7.36 (actual)
Steering angle: -25.010576674435715 (pred)	-7.36 (actual)
Steering angle: -25.007132551546736 (pred)	-7.36 (actual)
Steering angle: -25.712933564524587 (pred)	-7.36 (actual)
Steering angle: -25.68607896825539 (pred)	-7.36 (actual)
Steering angle: -26.511348527553228 (pred)	-7.36 (actual)
Steering angle: -22.21256236018331 (pred)	-7.36 (actual)
Steering angle: -24.85021920064945 (pred)	-7.36 (actual)
Steering angle: -21.094994855354777 (pred)	-7.36 (actual)
Steering angle: 10.32197311902003 (pred)	-7.36 (actual)
Steering angle: 9.517491240461124 (pred)	-7.36 (actual)
Steering angle: 0.7262503859094592 (pred)	-7.36 (actual)
Steering angle: 18.035177682668493 (pred)	-7.36 (actual)
Steering angle: 12.281467306984771 (pred)	-7.36 (actual)
Steering angle: -4.870554963170025 (pred)	-7.36 (actual)
Steering angle: -13.149984770

Steering angle: -9.481499558629743 (pred)	-7.26 (actual)
Steering angle: -19.760027551886616 (pred)	0.0 (actual)
Steering angle: -14.20596275186872 (pred)	-7.26 (actual)
Steering angle: -17.997382830896225 (pred)	-7.26 (actual)
Steering angle: -7.77722648805993 (pred)	-7.26 (actual)
Steering angle: -17.613621822189888 (pred)	-7.26 (actual)
Steering angle: -15.39860171963851 (pred)	-7.26 (actual)
Steering angle: -19.964815406223813 (pred)	-7.26 (actual)
Steering angle: -22.052744471341473 (pred)	-7.26 (actual)
Steering angle: -29.129346376040775 (pred)	-7.26 (actual)
Steering angle: -26.361782752655582 (pred)	-7.26 (actual)
Steering angle: -21.996303203135074 (pred)	-7.26 (actual)
Steering angle: -8.724861447659485 (pred)	-7.26 (actual)
Steering angle: 9.218718275597258 (pred)	-7.26 (actual)
Steering angle: 20.105051142715222 (pred)	-7.26 (actual)
Steering angle: 28.361662141635314 (pred)	-7.26 (actual)
Steering angle: 40.83773519845209 (pred)	0.0 (actual)
Steering angle: 51.66398459898

Steering angle: 86.49780251698662 (pred)	0.0 (actual)
Steering angle: 81.57068846484701 (pred)	-7.06 (actual)
Steering angle: 78.18263459876472 (pred)	-7.06 (actual)
Steering angle: 77.64083667304263 (pred)	-7.06 (actual)
Steering angle: 77.12045139036853 (pred)	-7.06 (actual)
Steering angle: 75.56682241081157 (pred)	-7.06 (actual)
Steering angle: 76.34827801413107 (pred)	-7.06 (actual)
Steering angle: 71.65299424280003 (pred)	-7.06 (actual)
Steering angle: 67.2570230212773 (pred)	-7.06 (actual)
Steering angle: 65.35886563960656 (pred)	-7.06 (actual)
Steering angle: 58.8124571692295 (pred)	-7.06 (actual)
Steering angle: 50.30883067142861 (pred)	-6.659999999999999 (actual)
Steering angle: 28.045274118901983 (pred)	-5.75 (actual)
Steering angle: 27.006322091420298 (pred)	-4.94 (actual)
Steering angle: 25.72202113121529 (pred)	-2.62 (actual)
Steering angle: 28.39525471952073 (pred)	-0.5 (actual)
Steering angle: 17.559463544713566 (pred)	1.92 (actual)
Steering angle: 38.05935426296889 (pre